# RMSA Experimental Notebook
## Análisis comparativo de algoritmos de RMSA en redes ópticas elásticas

Este notebook implementa First-Fit, Parcel-Fit y Sliding-Fit para comparar desempeño en topologías reales (Eurocore, NSFNet, UKNet, USNet).

### Instalación de dependencias

In [1]:
!pip3 install numpy pandas plotly networkx

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


### Importaciones necesarias

In [15]:
import json
import os
import re
import sys
from pathlib import Path
import time
import random

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

WORKSPACE_DIR = Path(os.getcwd()).resolve()
PROJECT_ROOT = WORKSPACE_DIR if (WORKSPACE_DIR / 'Rutas').exists() else WORKSPACE_DIR.parent
NOTEBOOKS_DIR = PROJECT_ROOT / 'notebooks'

for candidate in (NOTEBOOKS_DIR, PROJECT_ROOT):
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))

RUTAS_DIR = PROJECT_ROOT / 'Rutas'
RESULTS_DIR = PROJECT_ROOT / 'results'


### Importación de funciones del loader

In [16]:
try:
    from src.loader import obtener_enlaces_directos, crear_rutas_usuarios
except ModuleNotFoundError:
    from notebooks.src.loader import obtener_enlaces_directos, crear_rutas_usuarios

try:
    from src.algorithms.first_fit import find_first_fit_and_allocate
    from src.algorithms.parcel_fit import allocate_parcel_fit
    from src.algorithms.sliding_fit import allocate_sliding_fit
except ModuleNotFoundError:
    from notebooks.src.algorithms.first_fit import find_first_fit_and_allocate
    from notebooks.src.algorithms.parcel_fit import allocate_parcel_fit
    from notebooks.src.algorithms.sliding_fit import allocate_sliding_fit

### Configuración del experimento

In [17]:
ANALYSIS_MODE = "all"  # "selected" o "all"
SELECTED_TOPOLOGIES = [
    "TOP_1_ABILENE",
    "TOP_2_ANS",
    "TOP_3_ARNES",
]

DEMAND_SLOTS = 4            # Slots requeridos por cada par origen-destino
PARCEL_SIZE = 2             # Tamaño de parcela para Parcel-Fit

assert ANALYSIS_MODE in {"selected", "all"}, "ANALYSIS_MODE debe ser 'selected' o 'all'"


### Definición de métricas y funciones auxiliares

In [18]:
def fragmentation_metric(capacity):
    """Promedio de segmentos libres (contiguos) por enlace."""
    segments = []
    for row in capacity:
        free = (row == 0).astype(int)
        seg = 0
        inblock = False
        for value in free:
            if value == 1 and not inblock:
                seg += 1
                inblock = True
            elif value == 0:
                inblock = False
        segments.append(seg)
    return float(np.mean(segments)) if segments else 0.0


def spectrum_efficiency(capacity):
    """Proporción de ranuras utilizadas respecto al total disponible."""
    if capacity.size == 0:
        return 0.0
    return float(np.sum(capacity)) / capacity.size


TOPO_PATTERN = re.compile(r"^(TOP_(\d+)_.+?)_routes\.json$", re.IGNORECASE)


def discover_topologies(rutas_dir: Path):
    """Localiza archivos TOP_* y retorna metadatos ordenados por índice numérico."""
    rutas_dir = Path(rutas_dir)
    discovered = []
    for routes_file in rutas_dir.glob("TOP_*_routes.json"):
        match = TOPO_PATTERN.match(routes_file.name)
        if not match:
            continue
        base_name = match.group(1)
        topo_file = rutas_dir / f"{base_name}.json"
        if not topo_file.exists():
            continue
        discovered.append({
            "name": base_name,
            "numeric_id": int(match.group(2)),
            "routes_path": routes_file,
            "topology_path": topo_file,
        })
    return sorted(discovered, key=lambda item: item["numeric_id"])


def extract_slot_count(topo_path: Path, default_slots: int = 80) -> int:
    try:
        with open(topo_path, "r", encoding="utf-8") as handler:
            topo_data = json.load(handler)
        slot_values = {
            int(link.get("slots", default_slots))
            for link in topo_data.get("links", [])
            if "slots" in link
        }
        return min(slot_values) if slot_values else default_slots
    except FileNotFoundError:
        return default_slots


def load_topology_payload(meta: dict):
    with open(meta["routes_path"], "r", encoding="utf-8") as handler:
        routes_payload = json.load(handler)
    routes_df = pd.json_normalize(routes_payload["routes"])
    routes_df["paths"] = routes_df["paths"].apply(lambda paths: [paths[0]] if paths else [])
    enlaces = obtener_enlaces_directos(routes_df)
    rutas_usuarios = crear_rutas_usuarios(routes_df, enlaces)
    slots_per_link = extract_slot_count(meta["topology_path"])
    return {
        **meta,
        "alias": routes_payload.get("alias", meta["name"]),
        "routes_df": routes_df,
        "enlaces": enlaces,
        "rutas_usuarios": rutas_usuarios,
        "slots_per_link": slots_per_link,
        "n_connections": len(routes_df),
        "n_enlaces": len(enlaces),
    }


### Runner de experimentos RMSA

In [19]:
def run_rmsa_experiment(topology_name, routes_df, enlaces, rutas_usuarios, n_slots=80, n_demands=1000, seed=0):
    """
    Ejecuta los tres algoritmos RMSA sobre una topología y demandas aleatorias.
    Retorna un diccionario con métricas agregadas.
    """
    random.seed(seed)
    np.random.seed(seed)
    
    # Generar demandas aleatorias
    demand_choices = [1, 2, 3, 4, 6, 8, 10]
    demand_list = []
    for _ in range(n_demands):
        ridx = random.randrange(0, len(routes_df))
        size = random.choice(demand_choices)
        demand_list.append((ridx, size))
    
    results = {}
    
    # First-Fit
    capacity_ff = np.zeros((len(enlaces), n_slots), dtype=np.int8)
    blocked_ff = 0
    start_time = time.time()
    for (ridx, size) in demand_list:
        route_row = rutas_usuarios[ridx]
        _, ok = find_first_fit_and_allocate(capacity_ff, route_row, size)
        if not ok:
            blocked_ff += 1
    time_ff = time.time() - start_time
    results['first_fit'] = {
        'blocked': blocked_ff,
        'blocking_prob': blocked_ff / len(demand_list),
        'utilization': spectrum_efficiency(capacity_ff),
        'fragmentation': fragmentation_metric(capacity_ff),
        'time_s': time_ff
    }
    
    # Parcel-Fit
    capacity_pf = np.zeros((len(enlaces), n_slots), dtype=np.int8)
    blocked_pf = 0
    start_time = time.time()
    for (ridx, size) in demand_list:
        route_row = rutas_usuarios[ridx]
        _, ok = allocate_parcel_fit(capacity_pf, route_row, size, parcel_size=2)
        if not ok:
            blocked_pf += 1
    time_pf = time.time() - start_time
    results['parcel_fit'] = {
        'blocked': blocked_pf,
        'blocking_prob': blocked_pf / len(demand_list),
        'utilization': spectrum_efficiency(capacity_pf),
        'fragmentation': fragmentation_metric(capacity_pf),
        'time_s': time_pf
    }
    
    # Sliding-Fit
    capacity_sf = np.zeros((len(enlaces), n_slots), dtype=np.int8)
    blocked_sf = 0
    start_time = time.time()
    for (ridx, size) in demand_list:
        route_row = rutas_usuarios[ridx]
        _, ok = allocate_sliding_fit(capacity_sf, route_row, size)
        if not ok:
            blocked_sf += 1
    time_sf = time.time() - start_time
    results['sliding_fit'] = {
        'blocked': blocked_sf,
        'blocking_prob': blocked_sf / len(demand_list),
        'utilization': spectrum_efficiency(capacity_sf),
        'fragmentation': fragmentation_metric(capacity_sf),
        'time_s': time_sf
    }
    
    return results

### Carga de topologías de ejemplo (Eurocore, NSFNet)

In [20]:
available_topologies = discover_topologies(RUTAS_DIR)
if not available_topologies:
    raise RuntimeError(f"No se encontraron archivos TOP en {RUTAS_DIR}")

if ANALYSIS_MODE == "all":
    target_topologies = available_topologies
else:
    available_map = {meta["name"]: meta for meta in available_topologies}
    missing = [name for name in SELECTED_TOPOLOGIES if name not in available_map]
    if missing:
        print(f"Advertencia: no se encontraron {missing}. Se ignorarán.")
    target_topologies = [available_map[name] for name in SELECTED_TOPOLOGIES if name in available_map]

if not target_topologies:
    raise RuntimeError("No hay topologías seleccionadas para analizar.")

loaded_topos = {}
for meta in target_topologies:
    payload = load_topology_payload(meta)
    loaded_topos[payload["name"]] = payload
    print(
        f"{payload['name']}: alias={payload['alias']}, rutas={payload['n_connections']}, "
        f"enlaces={payload['n_enlaces']}, slots/link={payload['slots_per_link']}"
    )

print(f"\nTotal de topologías seleccionadas: {len(loaded_topos)}")


TOP_1_ABILENE: alias=ABILENE, rutas=110, enlaces=28, slots/link=320
TOP_2_ANS: alias=ANS, rutas=306, enlaces=50, slots/link=320
TOP_3_ARNES: alias=ARNES, rutas=272, enlaces=40, slots/link=320
TOP_4_ARPANET: alias=ARPANET, rutas=380, enlaces=64, slots/link=320
TOP_5_ATMNET: alias=ATMNET, rutas=420, enlaces=44, slots/link=320
TOP_6_BBNPLANET: alias=BBNPLANET, rutas=702, enlaces=56, slots/link=320
TOP_7_BEYONDTHENETWORK: alias=BEYONDTHENETWORK, rutas=812, enlaces=82, slots/link=320
TOP_8_BICS: alias=BICS, rutas=1056, enlaces=96, slots/link=320
TOP_9_BIZNET: alias=BIZNET, rutas=756, enlaces=64, slots/link=320
TOP_10_BREN: alias=BREN, rutas=90, enlaces=22, slots/link=320
TOP_11_CANARIE: alias=CANARIE, rutas=552, enlaces=66, slots/link=320
TOP_12_CANARIE: alias=CANARIE, rutas=342, enlaces=52, slots/link=320
TOP_13_CERNET: alias=CERNET, rutas=1260, enlaces=102, slots/link=320
TOP_14_CESNET: alias=CESNET, rutas=132, enlaces=34, slots/link=320
TOP_15_CLARANET: alias=CLARANET, rutas=210, enlaces

### Ejecución de experimentos en topologías cargadas

In [21]:
all_results = {}

for topo_name, topo_data in loaded_topos.items():
    print(f"\n=== Running experiments on {topo_name} ===")
    results = run_rmsa_experiment(
        topo_name,
        topo_data['routes_df'],
        topo_data['enlaces'],
        topo_data['rutas_usuarios'],
        n_slots=80,
        n_demands=500,  # reducido para desarrollo
        seed=0
    )
    all_results[topo_name] = results
    
    for alg_name, metrics in results.items():
        print(f"  {alg_name}:")
        print(f"    blocking_prob: {metrics['blocking_prob']:.6f}")
        print(f"    utilization: {metrics['utilization']:.6f}")
        print(f"    fragmentation: {metrics['fragmentation']:.2f}")
        print(f"    time: {metrics['time_s']:.3f}s")


=== Running experiments on TOP_1_ABILENE ===
  first_fit:
    blocking_prob: 0.576000
    utilization: 0.753571
    fragmentation: 2.82
    time: 0.082s
  parcel_fit:
    blocking_prob: 0.582000
    utilization: 0.779018
    fragmentation: 2.36
    time: 0.114s
  sliding_fit:
    blocking_prob: 0.576000
    utilization: 0.754018
    fragmentation: 3.68
    time: 0.108s

=== Running experiments on TOP_2_ANS ===
  first_fit:
    blocking_prob: 0.448000
    utilization: 0.706750
    fragmentation: 4.16
    time: 0.070s
  parcel_fit:
    blocking_prob: 0.444000
    utilization: 0.752750
    fragmentation: 4.02
    time: 0.127s
  sliding_fit:
    blocking_prob: 0.446000
    utilization: 0.708750
    fragmentation: 4.62
    time: 0.118s

=== Running experiments on TOP_3_ARNES ===
  first_fit:
    blocking_prob: 0.448000
    utilization: 0.706750
    fragmentation: 4.16
    time: 0.070s
  parcel_fit:
    blocking_prob: 0.444000
    utilization: 0.752750
    fragmentation: 4.02
    time: 0.12

### Visualización de resultados con plotly

In [22]:
# Preparar datos para graficar: número de bloqueos por topología y algoritmo
plot_data_blocking = []
for topo_name, topo_results in all_results.items():
    for alg_name, metrics in topo_results.items():
        plot_data_blocking.append({
            'Topología': topo_name,
            'Algoritmo': alg_name,
            'Número de bloqueos': metrics['blocked']
        })

df_blocking = pd.DataFrame(plot_data_blocking)
fig_blocking = px.line(
    df_blocking,
    x='Topología',
    y='Número de bloqueos',
    color='Algoritmo',
    markers=True,
    title='Número de bloqueos por topología y algoritmo RMSA',
    labels={'Número de bloqueos': 'Número de bloqueos'},
    log_y=True,
)
fig_blocking.show()


In [23]:
# Utilización de espectro
plot_data_util = []
for topo_name, topo_results in all_results.items():
    for alg_name, metrics in topo_results.items():
        plot_data_util.append({
            'Topología': topo_name,
            'Algoritmo': alg_name,
            'Utilización': metrics['utilization']
        })


df_util = pd.DataFrame(plot_data_util)
fig_util = px.line(
    df_util,
    x='Topología',
    y='Utilización',
    color='Algoritmo',
    markers=True,
    title='Utilización de espectro por topología y algoritmo',
    labels={'Utilización': 'Eficiencia de espectro'},
    log_y=True,
)
fig_util.show()


In [24]:
# Fragmentación
plot_data_frag = []
for topo_name, topo_results in all_results.items():
    for alg_name, metrics in topo_results.items():
        plot_data_frag.append({
            'Topología': topo_name,
            'Algoritmo': alg_name,
            'Fragmentación': metrics['fragmentation']
        })


df_frag = pd.DataFrame(plot_data_frag)
fig_frag = px.line(
    df_frag,
    x='Topología',
    y='Fragmentación',
    color='Algoritmo',
    markers=True,
    title='Fragmentación de espectro por topología y algoritmo',
    labels={'Fragmentación': 'Promedio de segmentos libres'}
)
fig_frag.show()


### Análisis de resultados

In [10]:
print("\n" + "="*60)
print("RESUMEN DE EXPERIMENTOS RMSA")
print("="*60)

for topo_name in sorted(all_results.keys()):
    print(f"\nTopología: {topo_name}")
    print("-" * 50)
    
    # Encontrar mejor algoritmo por métrica
    topo_results = all_results[topo_name]
    
    best_blocking = min(topo_results.items(), key=lambda x: x[1]['blocking_prob'])
    best_util = max(topo_results.items(), key=lambda x: x[1]['utilization'])
    best_frag = min(topo_results.items(), key=lambda x: x[1]['fragmentation'])
    
    print(f"  Menor bloqueo: {best_blocking[0]} ({best_blocking[1]['blocking_prob']:.6f})")
    print(f"  Mayor utilización: {best_util[0]} ({best_util[1]['utilization']:.6f})")
    print(f"  Menor fragmentación: {best_frag[0]} ({best_frag[1]['fragmentation']:.2f} segmentos)")

print("\n" + "="*60)


RESUMEN DE EXPERIMENTOS RMSA

Topología: Eurocore
--------------------------------------------------
  Menor bloqueo: sliding_fit (0.292000)
  Mayor utilización: parcel_fit (0.685054)
  Menor fragmentación: parcel_fit (2.37 segmentos)

Topología: NSFNet
--------------------------------------------------
  Menor bloqueo: sliding_fit (0.406000)
  Mayor utilización: parcel_fit (0.762216)
  Menor fragmentación: parcel_fit (3.32 segmentos)

Topología: UKNet
--------------------------------------------------
  Menor bloqueo: sliding_fit (0.442000)
  Mayor utilización: parcel_fit (0.491827)
  Menor fragmentación: first_fit (3.23 segmentos)

Topología: USNet
--------------------------------------------------
  Menor bloqueo: first_fit (0.340000)
  Mayor utilización: parcel_fit (0.467928)
  Menor fragmentación: first_fit (4.57 segmentos)



### Exportar resultados a CSV y JSON

In [ ]:
# Exportar como CSV
results_list = []
for topo_name, topo_results in all_results.items():
    for alg_name, metrics in topo_results.items():
        row = {'topology': topo_name, 'algorithm': alg_name}
        row.update(metrics)
        results_list.append(row)

df_results = pd.DataFrame(results_list)
results_dir = os.path.join(base_dir, 'results')
os.makedirs(results_dir, exist_ok=True)
csv_path = os.path.join(results_dir, 'rmsa_results.csv')
df_results.to_csv(csv_path, index=False)
print(f"Resultados exportados a {csv_path}")

# Exportar como JSON
json_path = os.path.join(results_dir, 'rmsa_results.json')
with open(json_path, 'w') as f:
    import json
    json.dump(all_results, f, indent=2)
print(f"Resultados exportados a {json_path}")

Resultados exportados a /Users/diego/Downloads/T2/RMSA_project/RMSA_project/results/rmsa_results.csv
Resultados exportados a /Users/diego/Downloads/T2/RMSA_project/RMSA_project/results/rmsa_results.json
